# Entrega 1 - Árboles de Decisión

### Grupo 9:
     - J. Gu       C.I 5.509.557-9
     - M. Nuñez    C.I 5.225.262-3
     - L. Pereira  C.I 5.268.309-4
     





## 1. Objetivo

El objetivo de este informe es implementar y evaluar un árbol de decisión basado en el algoritmo ID3 con soporte para atributos numéricos, incorporando un parámetro max_range_split, el cual indica la cantidad máxima de rangos en los que se pueden partir dichos atributos numéricos. 

Se realizará el preprocesamiento de datos numéricos, se entrenarán y evaluarán modelos usando tanto el ID3 modificado como los algoritmos DecisionTreeClassifier y RandomForestClassifier de scikit-learn, y se compararán los resultados obtenidos para cada enfoque.


## 2. Diseño

En esta sección se presentaran las decisiones tomadas a la hora de implementar el algoritmo, como también una descripción del procesamiento de datos realizado y los métodos de evaluación utilizados.


### 2.1 Preprocesamiento de datos
  Para nuestro laboratorio, comenzamos obteniendo el conjunto de datos y realizando una limpieza inicial. La primera acción fue eliminar la columna 0, que correspondía al identificador del paciente. Esta columna no aportaba información relevante para la predicción, ya que es simplemente un identificador único y no tiene correlación con las características de salud que queremos evaluar.
#### 2.1.1 Identificación de atributos categóricos y continuos
  A continuación, identificamos qué atributos eran categóricos y cuáles eran continuos. Esta diferenciación es crucial porque los atributos categóricos representan cualidades discretas, como tipos o categorías, mientras que los atributos continuos son valores numéricos que pueden tomar cualquier valor dentro de un rango. Tratar los atributos categóricos como continuos podría llevar a resultados erróneos. Para asegurarnos de que los atributos categóricos fueran tratados adecuadamente, los definimos explícitamente antes del entrenamiento del modelo. Esto fue necesario ya que todos los atributos del conjunto de datos brindados son numéricos, pero algunos de ellos son esencialmente categóricos (teniendo 0 ante una categoría, 1 ante otra, etc.). Al analizar el esquema del conjunto de datos brindado vimos que éste especificaba cuáles eran los atributos categóricos y cuáles no. Teniendo esto en cuenta decidimos que nuestra implementación del árbol de decisión recibiera como parámetro una lista con los índices de las columnas que eran categóricas. De esta forma diferenciamos entre los atributos categóricos y numéricos y actuamos en consecuencia dependiendo del caso.
#### 2.1.2 Enfoques de Preprocesamiento
  Adoptamos dos enfoques para manejar los atributos continuos y categóricos en el preprocesamiento:
  - Definición de Rangos dentro del Algoritmo: En el primer enfoque permitimos que el algoritmo determinara los mejores puntos de corte durante su ejecución. Esto se realizó durante el proceso de recursión del algoritmo, donde se evaluaban diferentes divisiones de los datos para maximizar la ganancia de información. Este enfoque es más dinámico y se adapta mejor a las características específicas de los datos de entrenamiento en cada pliegue. La implementación de este algoritmo que elige los mejores puntos de corte es explicada más adelante en la sección 2.2.1.
  
  - Discretización previa al entrenamiento: En el segundo enfoque, discretizamos los valores continuos antes del entrenamiento. La discretización implica dividir los valores continuos en intervalos o categorías, convirtiéndolos así en atributos discretos. Para determinar los mejores puntos de corte (intervalos) para la discretización, analizamos el conjunto de datos completo utilizando el mismo criterio que en el algoritmo de definición de rangos dentro de ID3. La diferencia con el otro enfoque radica en que hallamos los mejores puntos de corte una única vez antes de ingresar al algoritmo, por lo que el algoritmo interpreta a todos los atributos como categóricos. 
  
  
#### 2.1.3 Uso de OneHotEncoder para atributos categóricos
Para el manejo de atributos categóricos con los algoritmos de scikit-learn, utilizamos el OneHotEncoder. Este transformador es una herramienta que convierte cada categoría de un atributo categórico en una columna binaria separada. Si un atributo categórico tiene tres posibles valores, el OneHotEncoder creará tres columnas binarias (una para cada valor). Este enfoque es útil porque permite que los modelos interpreten correctamente los datos categóricos, ya que evita asignar valores ordinales (como 0, 1, 2) que podrían implicar un orden que no existe realmente. Esto mejora la calidad de las predicciones y evita sesgos inducidos por un mal manejo de los datos categóricos, y es necesario ya que sin él, los modelos de scikit-learn interpretan por defecto que todos los atributos son numéricos, por lo que el modelo que se genere no será del todo correcto.

#### 2.1.4 Partición del conjunto de datos
A la hora de decidir cómo ibamos a particionar el conjunto de datos para utilizar una parte para entrenamiento y otra para evaluación, decidimos utilizar la division en 5 partes del conjunto, ya que cómo método de validación pensamos utilizar validación cruzada. Punto que será mejor desarrollado en la sección de evaluación.


### 2.2 Consideraciones sobre algoritmo ID3
Para implementar el algoritmo ID3, nos basamos fuertemente en el algoritmo visto en el teórico del curso. En esta sección explicaremos las consideraciones que tuvimos a la hora de implementar el algoritmo ID3, y que escapan de la base del algoritmo, y como estas pueden afectar los resultados posteriores.

#### 2.2.1 Cálculo de ganancia y entropía
Los cálculos de ganancia y entropía son iguales a como fueron presentados en el curso. Las únicas consideraciónes que tuvimos fueron con el hiperparámetro max_range_split y con la decisión de retornar 0 ante algunos casos borde. 
Dado que nos interesaba observar los resultados de nuestra implementación con max_range_split igual a 2 y 3, no generalizamos en la implementación que max_range_split pueda tener cualquier valor, sino que solo consideramos esos casos. Esto se hizo ya que la generalización del hiperparámetro hacía que la implementación tuviera que tener varias complejidades extras que entendimos no serían necesarias para experimentar con el conjunto de datos dado.
Por otro lado, decidimos que al calcular la ganancia de un valor numérico, ante el caso de que alguno de los puntos de corte seleccionados dejara alguno de los subconjuntos vacío, retornaríamos 0 como ganancia. 
Luego, en el algoritmo se recorren todos los atributos y se elige aquel que genere máxima ganancia. Ante el caso en el que todos los atributos generen ganancia 0, se retorna el valor de etiqueta que más veces aparezca en esa rama y se finaliza la recursión, ya que si bajar un nivel más por el árbol no nos genera ganancia, deberíamos mantenerlo lo más genérico posible. 
Esto, por otro lado, implicitamente genera que, sin necesidad de ir eliminando los atributos que ya se utilizaron en el árbol, no estemos eligiendo más de una vez en el árbol un atributo categórico (ya que su ganancia luego es 0 y no sería seleccionado), pero si lo permite para atributos numéricos utilizando distintas particiones al profundizar en las ramas. El hecho de que los atributos numéricos puedan ser reutilizados como atributos en el árbol tiene que tomarse con cuidado ya que podría generar, dependiendo de las características del conjunto de datos, una rama infinita en la que se particione por dicho atributo numérico infinitas veces. En el caso particular de el conjunto de datos utilizado, esto no sucedió y por ende solamente se registra su posibilidad de ocurrencia pero no fue tenida en cuenta su resolución en la implementación.

#### 2.2.2 ¿Qué puntos de corte utilizar ante atributos numéricos?
Como tercer consideración, tuvimos que ver cómo ibamos a seleccionar los puntos de corte para el caso de la división de los atributos numéricos. Como fue discutido en clase, se podría identificar los posibles puntos de corte ordenando el conjunto de datos de acuerdo a ese atributo y verificando en qué valores la etiqueta cambia su valor de positivo a negativo o viceversa. Al haber un cambio de valor, el punto medio entre los dos valores del atributo en los que se da ese cambio sería un posible punto de corte. Esto sin embargo puede llegar a generar múltiples puntos de corte, mientras que nuestro algoritmo solamente permitía 1 o 2 puntos (dependiendo del valor que se le diera a max_range_split). Para solventar esta situación decidimos tomar todas las combinaciones de puntos de corte, identificadas previamente, de a 1 y 2 puntos, calcular la ganancia que generaría utilizarlos efectivamente como puntos de corte, y finalmente quedarnos con el/los punto/s de corte que nos generen ganancia máxima. Al realizar esta implementación tuvimos en cuenta que podía suceder que los tiempos de ejecución fueran demasiado altos, ya que el tomar todas las combinaciones y calcularles la ganancia es algo bastante ineficiente dado el orden que tiene, pero, al probar la implementación con el conjunto de datos brindado, descubrimos que igualmente se ejecutaba en pocos minutos, por lo que decidimos mantenerlo de esa forma. No obstante, entendemos que dado otro conjunto de datos con mayor cantidad de atributos y/o mayor cantidad de filas, esta opción podría tardar demasiado tiempo, en ese caso se podría optar por soluciones como tomar N combinaciones de los datos para calcularles la ganancia, o elegir de manera azarosa la cantidad de puntos de corte que se quiera dados los puntos de corte encontrados previamente.



## 2.3 Evaluación
Se probará los distintos algoritmos utilizando el conjunto de datos «AIDS Clinical Trials Group Study 175».
### 2.3.1 Métricas utilizadas para la evaluación de la solución
En este estudio, la métrica principal utilizada para evaluar el rendimiento de los modelos fue la precisión. La precisión mide el porcentaje de predicciones correctas sobre el total de predicciones realizadas. Es una métrica común en problemas de clasificación, especialmente cuando las clases están balanceadas. Se define como: 
$$
\text{Precisión} = \frac{\text{Número de predicciones correctas}}{\text{Número total de predicciones}}
$$
- La precisión es una métrica fácil de interpretar y proporciona una buena visión general de qué tan bien está desempeñándose el modelo en términos generales.

- Desviación Estándar de la Precisión: Además de la precisión promedio, también calculamos la desviación estándar de la precisión. La desviación estándar mide la variabilidad o dispersión de las puntuaciones de precisión a través de los diferentes pliegues de la validación cruzada. Una desviación estándar baja sugiere que el modelo es consistente en diferentes subconjuntos de los datos, mientras que una desviación estándar alta podría indicar que el rendimiento del modelo es inestable o depende en gran medida de la división particular de los datos.

### 2.3.2 Construcción de los conjuntos de entrenamiento, ajuste y evaluación
Para evaluar la efectividad de los modelos de aprendizaje automático, utilizamos la técnica de validación cruzada con 5 pliegues. Este método implica dividir el dataset en cinco subconjuntos aproximadamente iguales. En cada iteración, uno de los subconjuntos se usa como conjunto de prueba, mientras que los otros cuatro se utilizan para entrenar el modelo. Este proceso se repite cinco veces, de modo que cada subconjunto se utiliza una vez como conjunto de prueba.

### 2.3.3 ¿Por qué validación cruzada?
Optar por la validación cruzada de 5 pliegues nos permitió utilizar todo el dataset para el entrenamiento y la evaluación, lo cual es beneficioso cuando se trabaja con conjuntos de datos que no son extremadamente grandes. Esta técnica también ayuda a mitigar el riesgo de sobreajuste al proporcionar una evaluación más completa del rendimiento del modelo en diferentes particiones del dataset.

## 3. Experimentación

En las siguientes tablas se presentan los resultados obtenidos, mostrando la precisión promedio y la desviación estándar de los cinco pliegues generados por la validación cruzada para los diferentes algoritmos evaluados. En lugar de detallar los resultados individuales de cada pliegue, solo se exponen los promedios y las desviaciones estándar calculadas.

La Tabla 1 muestra los resultados obtenidos para el algoritmo ID3 con diferentes configuraciones de max_range_split y utilizando datos preprocesados, mientras que la Tabla 2 incluye los resultados de DecisionTreeClassifier, RandomForestClassifier y el algoritmo ID3 sobre datos preprocesados en tiempo de ejecución.

<table>
  <tr>
    <th>Algoritmo</th>
    <th>Precisión promedio (%)</th>
    <th>Desviación estándar (%)</th>
  </tr>
  <tr>
    <td>ID3 (max_range_split = 2)</td>
    <td>83.50</td>
    <td>1.17</td>
  </tr>    
  <tr>
    <td>ID3 (max_range_split = 3)</td>
    <td>83.54</td>
    <td>1.14</td>
  </tr>    
  <caption>Tabla 1 - Resultados de ID3 con datos preprocesados</caption>
</table>


<table>
  <tr>
    <th>Algoritmo</th>
    <th>Precisión promedio (%)</th>
    <th>Desviación estándar (%)</th>
  </tr>
  <tr>
    <td>DecisionTreeClassifier (criterion = 'gini')</td>
    <td>85.46</td>
    <td>1.55</td>
  </tr>    
  <tr>
    <td>DecisionTreeClassifier (criterion = 'entropy')</td>
    <td>83.50</td>
    <td>1.07</td>
  </tr>    
  <tr>
    <td>DecisionTreeClassifier (criterion = 'log_loss')</td>
    <td>83.50</td>
    <td>1.07</td>
  </tr>
  <tr>
    <td>RandomForestClassifier (criterion = 'gini')</td>
    <td>89.15</td>
    <td>1.22</td>
  </tr>    
  <tr>
    <td>RandomForestClassifier (criterion = 'entropy')</td>
    <td>89.20</td>
    <td>1.34</td>
  </tr>    
  <tr>
    <td>RandomForestClassifier (criterion = 'log_loss')</td>
    <td>89.20</td>
    <td>1.34</td>
  </tr>
  <tr style="font-weight:bold">
    <td>ID3 (max_range_split = 2)</td>
    <td>84.15</td>
    <td>1.21</td>
  </tr>    
  <tr style="font-weight:bold">
    <td>ID3 (max_range_split = 3)</td>
    <td>84.38</td>
    <td>1.34</td>
  </tr>  
  <caption>Tabla 2 - Resultados con datos procesados durante la ejecución</caption>
</table>


## 4. Conclusión

Una breve conclusión del trabajo realizado. Por ejemplo: 
- ¿cuándo se dieron los mejores resultados del jugador?
- ¿encuentra alguna relación con los parámetros / oponentes/ atributos elegidos?
- ¿cómo mejoraría los resultados?

En este laboratorio, hemos implementado y evaluado un árbol de decisión basado en el algoritmo ID3, extendido para soportar atributos numéricos mediante el hiperparámetro max_range_split. También hemos comparado los resultados obtenidos con otros modelos de árboles de decisión proporcionados por scikit-learn, como DecisionTreeClassifier y RandomForestClassifier.

### 4.1 Mejores Resultados:
Los mejores resultados en términos de precisión se obtuvieron con el modelo RandomForestClassifier, particularmente utilizando el criterio entropy y log_loss, alcanzando una precisión promedio cercana al 89.20% y con una desviación estándar relativamente baja, lo que indica una buena estabilidad del modelo. En contraste, el algoritmo ID3 obtuvo su mejor precisión promedio de 84.38% cuando se utilizó max_range_split igual a 3, aunque con una desviación estándar un poco más alta que las de los modelos de RandomForestClassifier, lo cual sugiere una mayor variabilidad en el desempeño del modelo ID3.

### 4.2 Relaciones Observadas:
Al analizar los resultados, se observó que el valor del hiperparámetro max_range_split influyó en la precisión del algoritmo ID3, aunque la mejora entre max_range_split = 2 y max_range_split = 3 fue marginal. Esto sugiere que, para este conjunto de datos específico, un mayor número de puntos de corte en los atributos numéricos no necesariamente lleva a una mejora significativa en el rendimiento. Además, los modelos de scikit-learn, que incluyen técnicas de ensemble como el RandomForestClassifier, parecen beneficiarse de su capacidad para manejar la variabilidad y sobreajuste mejor que un solo árbol de decisión como ID3.

### 4.3 Posibles Mejoras:
Para mejorar los resultados, podrían considerarse las siguientes acciones:

Optimización del Hiperparámetro max_range_split: Aunque solo se evaluaron los valores 2 y 3, podría ser útil explorar otros valores de max_range_split.

Enriquecimiento del Preprocesamiento: Explorar otros métodos de preprocesamiento de datos, como normalización o escalado de los atributos numéricos, podría mejorar el rendimiento de los modelos. También se podría considerar el uso de técnicas de reducción de dimensionalidad como PCA para ver si se obtiene un modelo más eficiente y preciso.

Ajuste de Modelos con scikit-learn: Se podrían ajustar más hiperparámetros en los modelos de scikit-learn, como el número de árboles en RandomForestClassifier o la profundidad máxima en DecisionTreeClassifier, para maximizar la precisión y minimizar la desviación estándar.

Incorporación de Validación Cruzada: Aunque ya se utilizó validación cruzada en este laboratorio, implementar técnicas de validación más exhaustivas, como la búsqueda de hiperparámetros con validación cruzada en cada iteración, podría optimizar aún más los resultados.

### 4.4 Conclusión General:
El trabajo realizado permitió una comprensión profunda de cómo los distintos modelos de árboles de decisión pueden ser aplicados y ajustados a un conjunto de datos real. Aunque el algoritmo ID3 modificado mostró un desempeño aceptable, los modelos más avanzados de scikit-learn demostraron ser más robustos y precisos en este contexto, subrayando la importancia de considerar técnicas más complejas en problemas de clasificación cuando se dispone de datos suficientemente grandes y variados.